In [1]:
import os

In [2]:
%pwd

'c:\\Users\\rushi\\AI with DL Suraj Kale\\MLops project\\project-AI_DL-end-to-end-main\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\rushi\\AI with DL Suraj Kale\\MLops project\\project-AI_DL-end-to-end-main'

In [5]:
# update entity

In [6]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [7]:
from src.cnnClassifier.utils.comman import read_yaml,create_directories
from src.cnnClassifier.components import * 

In [8]:
from src.cnnClassifier.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH


In [9]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [10]:
import os
import zipfile
import gdown

In [11]:
from src.cnnClassifier import logger
from src.cnnClassifier.utils.comman import get_size

In [12]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    
    def download_file(self)-> str:
        '''
        Fetch data from the url
        '''

        try: 
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]
            prefix = 'https://drive.google.com/drive/folders/1MQg2jzglfbpNcwQJGthQCOmCCjYTI9QP?usp=sharing'
            gdown.download(prefix+file_id,zip_download_dir)

            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")

        except Exception as e:
            raise e
        
    

    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [13]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-04-07 23:59:12,968: INFO: comman: yaml file: config\config.yaml loaded successfully]
[2025-04-07 23:59:12,975: INFO: comman: yaml file: params.yaml loaded successfully]
[2025-04-07 23:59:12,978: INFO: comman: created directory at: artifacts]
[2025-04-07 23:59:12,980: INFO: comman: created directory at: artifacts/data_ingestion]
[2025-04-07 23:59:12,984: INFO: 1930546134: Downloading data from https://drive.google.com/drive/folders/1MQg2jzglfbpNcwQJGthQCOmCCjYTI9QP?usp=sharing into file artifacts/data_ingestion/data.zip]


c:\Users\rushi\anaconda3\envs\dlenv\Lib\site-packages\gdown\parse_url.py:48: UserWarning: You specified a Google Drive link that is not the correct link to download a file. You might want to try `--fuzzy` option or the following url: https://drive.google.com/uc?id=None
  warnings.warn(
Downloading...
From: https://drive.google.com/drive/folders/1MQg2jzglfbpNcwQJGthQCOmCCjYTI9QP?usp=sharingfolders
To: c:\Users\rushi\AI with DL Suraj Kale\MLops project\project-AI_DL-end-to-end-main\artifacts\data_ingestion\data.zip
1.23MB [00:01, 971kB/s]

[2025-04-07 23:59:16,831: INFO: 1930546134: Downloaded data from https://drive.google.com/drive/folders/1MQg2jzglfbpNcwQJGthQCOmCCjYTI9QP?usp=sharing into file artifacts/data_ingestion/data.zip]


BadZipFile: File is not a zip file